In [8]:
!pip install torch torchvision torchsummary streamlit pyngrok


In [9]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [2]:
# prompt: mount g drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
"""%%writefile app.py
import torch
from torchvision import models
import streamlit as st
import os
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader
from torchsummary import summary
import streamlit as st
from pyngrok import ngrok
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, utils, models
from collections import OrderedDict
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


# Set Web App title
st.title('PNEUMONIA Image Classification')

#Set Header
st.header('Please upload the X-ray Image here: ')

#Upload a file
file=st.file_uploader('', type=['jpeg', 'jpg', 'png'])
##load model
# Assuming the model is saved as 'model.pth' in your Google Drive
model_path = '/content/drive/My Drive/ZikryV3_model.pth'  # Replace with your actual path

# Load the state dictionary
state_dict = torch.load(model_path)

# Determine the number of classes from the saved state dictionary
num_classes = state_dict['fc.weight'].shape[0]  # Get the number of output features in the 'fc' layer

# Create a ResNet model with the correct number of classes
model = models.resnet18(num_classes=num_classes)  # Set num_classes

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Now you can set the model to eval mode
model.eval()

##class Label/classname
with open('/content/drive/My Drive/labels.txt','r') as f:
    class_names = [a[:-1].split(' ')[1] for a in f.readlines()]
    f.close()
print(class_names)

##Display image
if file is not None:
   image = Image.open(file).convert('RGB')
   st.image(image, caption='Uploaded Image.', use_column_width=True)

   #classify Image


   #write Classification
   st.write("## {}".format(class_names))
   st.write("### score: {}".format(conf_score))

# Define classify function
def classify(image, model, class_name):
    # Convert image to (224, 224)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = transform(image).unsqueeze(0).to(device)

    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        conf_score = torch.nn.functional.softmax(outputs, dim=1)[0][predicted].item()
        class_name = class_names[predicted.item()]

    return class_name, conf_score"""




Overwriting app.py


In [27]:
%%writefile app.py
import torch
from torchvision import models
import streamlit as st
import os
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader
from torchsummary import summary
import streamlit as st
from pyngrok import ngrok
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, utils, models
from collections import OrderedDict
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set Web App title
st.title('PNEUMONIA Image Classification')

#Set Header
st.header('Please upload the X-ray Image here: ')

#Upload a file
file=st.file_uploader('', type=['jpeg', 'jpg', 'png'])

##load model
# Assuming the model is saved as 'model.pth' in your Google Drive
model_path = '/content/drive/My Drive/ZikryV3_model.pth'  # Replace with your actual path

# Load the state dictionary
state_dict = torch.load(model_path)

# Determine the number of classes from the saved state dictionary
num_classes = state_dict['fc.weight'].shape[0]  # Get the number of output features in the 'fc' layer

# Create a ResNet model with the correct number of classes
model = models.resnet18(num_classes=num_classes)  # Set num_classes

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Now you can set the model to eval mode
model.eval()

##class Label/classname
with open('/content/drive/My Drive/labels.txt','r') as f:
    class_names = [a[:-1].split(' ')[1] for a in f.readlines()]
    f.close()
print(class_names)

##Display image
if file is not None:
   image = Image.open(file).convert('RGB')
   st.image(image, caption='Uploaded Image.', use_column_width=True)

   # Convert image to (224, 224)
   transform = transforms.Compose([
       transforms.Resize((224, 224)),
       transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   ])
   image = transform(image).unsqueeze(0).to(device)

   # Make prediction
   model.eval()
   with torch.no_grad():
       outputs = model(image)
       _, predicted = torch.max(outputs, 1)
       conf_score = torch.nn.functional.softmax(outputs, dim=1)[0][predicted].item()
       class_name = class_names[predicted.item()]

   #write Classification
   st.write("## {}".format(class_name))
   st.write("### score: {}".format(conf_score))



Overwriting app.py


In [ ]:
"""
import torch
import streamlit as st
import os
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

# Set Web App title
st.title('PNEUMONIA Image Classification')

# Set Header
st.header('Please upload the X-ray Image here: ')

# Upload a file
file = st.file_uploader('', type=['jpeg', 'jpg', 'png'])

# Load model
model_path = '/content/drive/My Drive/ZikryV3_model.pth'  # Replace with your actual path
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

try:
    model = models.resnet18(num_classes=2)  # Set num_classes
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
except Exception as e:
    st.error(f"Error loading model: {e}")

# Load class names
class_names = ['Normal', 'Pneumonia']  # Hardcode class names

# Display image
if file is not None:
    image = Image.open(file).convert('RGB')
    st.image(image, caption='Uploaded Image.', use_column_width=True)

    # Define classify function
    def classify(image, model, class_names):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        image = transform(image).unsqueeze(0).to(device)

        # Make prediction
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            conf_score = torch.nn.functional.softmax(outputs, dim=1)[0][predicted].item()
            class_name = class_names[predicted.item()]

        return class_name, conf_score

    # Classify image
    class_name, conf_score = classify(image, model, class_names)

    # Write classification
    st.write("## {}".format(class_name))
    st.write("### score: {:.2f}%".format(conf_score * 100))

    # Add confidence threshold
    if conf_score < 0.5:
        st.write("### Low confidence score. Please check the image again.")"""

'import torch\nimport streamlit as st\nimport os\nfrom torchvision import models\nimport torchvision.transforms as transforms\nfrom PIL import Image\nimport matplotlib.pyplot as plt\n\n# Set Web App title\nst.title(\'PNEUMONIA Image Classification\')\n\n# Set Header\nst.header(\'Please upload the X-ray Image here: \')\n\n# Upload a file\nfile = st.file_uploader(\'\', type=[\'jpeg\', \'jpg\', \'png\'])\n\n# Load model\nmodel_path = \'/content/drive/My Drive/Zikry_model.pth\'  # Replace with your actual path\ndevice = torch.device(\'cuda:0\' if torch.cuda.is_available() else \'cpu\')\n\ntry:\n    model = models.resnet18(num_classes=2)  # Set num_classes\n    model.load_state_dict(torch.load(model_path, map_location=device))\n    model.eval()\nexcept Exception as e:\n    st.error(f"Error loading model: {e}")\n\n# Load class names\nclass_names = [\'Normal\', \'Pneumonia\']  # Hardcode class names\n\n# Display image\nif file is not None:\n    image = Image.open(file).convert(\'RGB\')\n    s

In [22]:
!wget -q -O - ipv4.icanhazip.com

35.189.180.251


In [32]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://172.28.0.12:8504
  External URL: http://35.189.180.251:8504

your url is: https://vast-garlics-sing.loca.lt
  Stopping...
^C


In [ ]:
#!ngrok authtoken 2kjOeavv3Irk6qSedBbkfvDqCPW_5e37YF1xEsKwGaQ2ZHmzu

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Cell 2: Wait for app to start
"""import time
time.sleep(10)"""

In [ ]:
#!http 8501

/bin/bash: line 1: http: command not found
